In [13]:
import cv2 as cv
import numpy as np
import os
from PIL import Image


In [14]:
def get_br(mat):
    red=mat[:,:,2]*0.222
    green=mat[:,:,1]*0.707
    blue=mat[:,:,0]*0.071
    mat1=np.zeros_like(mat[:,:,0])
    mat1 = red+green+blue
    return mat1

# def crop_image(mat,img_hight,x_min):
#    x_max,y_min=0,0
#    if x_min>mat.shape[1]:
#       x_min = mat.shape[1]-1
#    while sum(mat[y_min,x_max,:])==0:
#       y_min+=1
#    img = mat[y_min+1:y_min+img_hight+1, :x_min,:]
#    x_min-=1
#    while np.sum(img[0,x_min,:])==0:
#       x_min-=1
#    img = img[:, :x_min,:]
#    return img

# def crop_image_left(mat,img_hight,x_min):
#    y_min=0
#    x_max = mat.shape[1]
#    if x_min>mat.shape[1]:
#       x_min=0
#    else:
#       x_min=mat.shape[1] - x_min
#    while np.sum(mat[:,x_max-1,:])==0:
#       x_max-=1
#    while sum(mat[y_min,x_max-1,:])==0:
#       y_min+=1   
#    img = mat[y_min+1:y_min+img_hight+1, x_min:,:]
#    x_min=0
#    while np.sum(img[0,x_min,:])==0:
#       x_min+=1
#    img = img[:, x_min:,:]
#    return img

def crop_image(mat, hight, x_max,params):
    crop_mode,crop_width,crop_hight = params
    crop_hight=int(crop_hight)
    crop_width =int(crop_width)

    if crop_mode:
        img = mat[crop_hight:crop_hight+hight, crop_width:, :]
        if img.shape[1]>x_max:
            img = img[:,img.shape[1]-x_max:,:]
    else:
        img = mat[crop_hight:crop_hight+hight,: mat.shape[1]-crop_width, :]
        if img.shape[1]>x_max:
            img = img[:,:x_max,:]
    return img

def warpImagesUnderControl(img1, img2, H):
  rows1, cols1 = img1.shape[:2]
  rows2, cols2 = img2.shape[:2]


  list_of_points_1 = np.float32([[0,0], [0, rows1],[cols1, rows1], [cols1, 0]]).reshape(-1, 1, 2)
  temp_points = np.float32([[0,0], [0,rows2], [cols2,rows2], [cols2,0]]).reshape(-1,1,2)

  list_of_points_2 = cv.perspectiveTransform(temp_points, H)
  k=0
  if np.max(np.abs(list_of_points_2))>5000:
    list_of_points_2 = np.abs(list_of_points_2)
    while np.max(np.abs(list_of_points_2))>5000:
      k+=1
      if list_of_points_2[0,0,0] >5000 or list_of_points_2[1,0,0] >5000:
        temp_points = np.float32([[k,0], [k,rows2], [cols2,rows2], [cols2,0]]).reshape(-1,1,2)
      else:
        temp_points = np.float32([[0,0], [0,rows2], [cols2-k,rows2], [cols2-k,0]]).reshape(-1,1,2)
      list_of_points_2 = cv.perspectiveTransform(temp_points, H)

  list_of_points = np.concatenate((list_of_points_1, list_of_points_2), axis=0)

  [x_min, y_min] = np.int32(list_of_points.min(axis=0).ravel() - 0.5)
  [x_max, y_max] = np.int32(list_of_points.max(axis=0).ravel() + 0.5)

  crop_mode =True

  if list_of_points_2[0,0,0]>=0:
    img2=img2[:,:img2.shape[1]-k,:]
    crop_mode=False
    crop_width = np.abs(list_of_points_2[2,0,0]-list_of_points_2[3,0,0])
    crop_hight = np.abs(list_of_points_2[3,0,1])
  else:
    img2=img2[:,:img2.shape[1]-k,:]
    crop_mode=True
    crop_width = np.abs(list_of_points_2[0,0,0]-list_of_points_2[1,0,0])
    crop_hight = np.abs(list_of_points_2[0,0,1])


  translation_dist = [-x_min, -y_min]


  H_translation = np.array([[1, 0, translation_dist[0]], [0, 1, translation_dist[1]], [0, 0, 1]])


  output_img = cv.warpPerspective( img2, H_translation.dot(H), (x_max-x_min, y_max-y_min),borderValue=0)
  output_img[translation_dist[1]:rows1+translation_dist[1], translation_dist[0]:cols1+translation_dist[0]] = img1

  return output_img, [crop_mode,crop_width,crop_hight]


In [15]:
def merge_images(img1,img2):
    result = 0
    orb = cv.ORB_create(nfeatures=2000)

    keypoints1, descriptors1 = orb.detectAndCompute(img1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(img2, None)

    bf = cv.BFMatcher_create(cv.NORM_HAMMING)
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)

    good = []
    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good.append(m)

    MIN_MATCH_COUNT = 10

    if len(good) > MIN_MATCH_COUNT:
        src_pts = np.float32([ keypoints1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
        dst_pts = np.float32([ keypoints2[m.trainIdx].pt for m in good]).reshape(-1,1,2)

        M, _ = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)

        result,crop_params = warpImagesUnderControl(img2, img1, M)

        result = crop_image(result,img1.shape[0],int((img1.shape[1]+img2.shape[1]-1)*1.2),crop_params)
    
    return result

def pil_to_cv(img):
    pil_image = Image.open(img)
    img= np.array(pil_image)
    img = np.copy(img[:,:,::-1])
    return img

def brightness_match(img1,img2,good,keypoints1):
    img1_br = get_br(img1)
    print(img1_br.shape)
    img2_br = get_br(img2)
    br1=0
    br2=0
    for m in good:
        x,y = map(int,keypoints1[m.queryIdx].pt)
        br1+=img1_br[y,x]
        x,y = map(int,keypoints1[m.trainIdx].pt)
        br2+=img2_br[y,x]
    img1=np.float64(img1)-(br1/len(good)-br2/len(good))
    img1[img1>255] = 255
    img1[img1<0] =0
    print(img1_br.shape)
    img1 = img1.astype('uint8')


In [16]:
test_folders = ['building1','building2','building3']
for folder in test_folders:
    path_to_dir = os.path.abspath(folder)+'\\'
    imgs = os.listdir(path_to_dir)
    if len(imgs)%2==0:
        left=merge_images(cv.imread(path_to_dir+imgs[len(imgs)//2]),cv.imread(path_to_dir+imgs[len(imgs)//2-1]))
        right=left.copy()
    else:
        left=cv.imread(path_to_dir+imgs[len(imgs)//2])
        right=left.copy()
    for i in range(len(imgs)//2):
        left = merge_images(cv.imread(path_to_dir+imgs[len(imgs)//2-i-1]),left)
        right = merge_images(cv.imread(path_to_dir+imgs[len(imgs)//2+i+1]),right)
    cv.imwrite(folder+f'_complete.jpg',merge_images(left,right))

